<h1> Parameter Tunning with Normalization
    
    RandomizedCV for: hidden layers, 
    neurons, optimizer, kernel initializer and bias initializer

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import Normalizer;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# Import data; set X and y; fill nan values and split in test and training  data:

In [2]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018060000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

# Fill nan values (BEFORE OR AFTER TEST, TRAIN SPLIT!!!)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Apply feature scaling:

In [3]:
sc_X = Normalizer(norm = 'l2') 
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [4]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
import keras.optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Using TensorFlow backend.


# Create regressor:

In [5]:
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 1, n_neurons = 11, optimizer = 'RMSprop', kernel_initializer="he_normal",
    bias_initializer= initializers.Ones()):
    model = Sequential()
    model.add(Dense(output_dim = n_neurons, 
                    input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dense(output_dim = 1, 
                    activation = 'linear'))
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

# Apply Randomized tunning for the whole ANN:

In [6]:
regressor = KerasRegressor(build_fn = regressor_tunning)

# Dictionary to include the parameters
parameters = {'n_hidden': [1, 2, 3, 4, 6],
              'n_neurons': np.arange(10,100, 5),
              'bias_initializer':[initializers.Zeros(),
                                 initializers.Ones()],
              'kernel_initializer': ['glorot_uniform',
                                     'he_normal',
                                     'he_uniform'],
              'optimizer': [keras.optimizers.RMSprop(), 
                            keras.optimizers.Adam(), 
                            keras.optimizers.Nadam(),
                            keras.optimizers.Adamax()]
               }

tscv = TimeSeriesSplit(n_splits = 4)

# add some early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mse', patience = 15)

rnd_search_cv = RandomizedSearchCV(estimator = regressor,
                                   param_distributions = parameters,
                                   scoring = 'neg_mean_squared_error',
                                   n_iter = 10,
                                   n_jobs = -1,
                                   cv = tscv)

# checkpoints:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")

rnd_search_cv.fit(X_train, y_train, batch_size = 32, epochs = 60, callbacks=[early_stopping])


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=15)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
1850/1850 [==============================] - 1s 647us/step - loss: 11580.9210 - mse: 11580.9180 - mae: 102.4698
Epoch 2/60
1850/1850 [==============================] - 0s 49us/step - loss: 2090.5998 - mse: 2090.5996 - mae: 33.0774
Epoch 3/60
1850/1850 [==============================] - 0s 48us/step - loss: 1038.2455 - mse: 1038.2454 - mae: 22.3205
Epoch 4/60
1850/1850 [==============================] - 0s 48us/step - loss: 1037.0094 - mse: 1037.0094 - mae: 22.4452
Epoch 5/60
1850/1850 [==============================] - 0s 48us/step - loss: 1033.4068 - mse: 1033.4069 - mae: 22.3420
Epoch 6/60
1850/1850 [==============================] - 0s 48us/step - loss: 1032.3506 - mse: 1032.3506 - mae: 22.2860
Epoch 7/60
1850/1850 [==============================] - 0s 48us/step - loss: 1031.3163 - mse: 1031.3162 - mae: 22.2316
Epoch 8/60
1850/1850 [==============================] - 0s 48us/step - loss: 1026.4194 - mse: 1026.4194 - mae: 22.1541
Epoch 9/60
1850/1850 [======================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=15)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
3699/3699 [==============================] - 1s 301us/step - loss: 6362.1636 - mse: 6362.1650 - mae: 64.2643
Epoch 2/60
3699/3699 [==============================] - 0s 48us/step - loss: 1139.5512 - mse: 1139.5511 - mae: 22.7663
Epoch 3/60
3699/3699 [==============================] - 0s 48us/step - loss: 1138.0285 - mse: 1138.0282 - mae: 22.8165
Epoch 4/60
3699/3699 [==============================] - 0s 48us/step - loss: 1132.0602 - mse: 1132.0604 - mae: 22.6361
Epoch 5/60
3699/3699 [==============================] - 0s 49us/step - loss: 1131.5117 - mse: 1131.5118 - mae: 22.6547
Epoch 6/60
3699/3699 [==============================] - 0s 49us/step - loss: 1130.6898 - mse: 1130.6898 - mae: 22.6984
Epoch 7/60
3699/3699 [==============================] - 0s 49us/step - loss: 1134.5932 - mse: 1134.5933 - mae: 22.7119
Epoch 8/60
3699/3699 [==============================] - 0s 48us/step - loss: 1132.2801 - mse: 1132.2803 - mae: 22.7014
Epoch 9/60
3699/3699 [=========================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=15)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


...

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=40)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
5548/5548 [==============================] - 1s 224us/step - loss: 3962.1714 - mse: 3962.1709 - mae: 41.5247
Epoch 2/60
5548/5548 [==============================] - 0s 53us/step - loss: 1717.1934 - mse: 1717.1937 - mae: 25.2393
Epoch 3/60
5548/5548 [==============================] - 0s 53us/step - loss: 1708.4010 - mse: 1708.4010 - mae: 25.2463
Epoch 4/60
5548/5548 [==============================] - 0s 53us/step - loss: 1706.4492 - mse: 1706.4493 - mae: 25.1905
Epoch 5/60
5548/5548 [==============================] - 0s 53us/step - loss: 1703.1908 - mse: 1703.1909 - mae: 25.1370
Epoch 6/60
5548/5548 [==============================] - 0s 53us/step - loss: 1697.5003 - mse: 1697.5001 - mae: 25.0364
Epoch 7/60
5548/5548 [==============================] - 0s 53us/step - loss: 1698.0452 - mse: 1698.0458 - mae: 25.2418
Epoch 8/60
5548/5548 [==============================] - 0s 53us/step - loss: 1692.4848 - mse: 1692.4850 - mae: 25.1184
Epoch 9/60
5548/5548 [=========================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=40)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
7397/7397 [==============================] - 1s 191us/step - loss: 3300.2270 - mse: 3300.2258 - mae: 37.6871
Epoch 2/60
7397/7397 [==============================] - 0s 53us/step - loss: 1654.9439 - mse: 1654.9441 - mae: 25.5746
Epoch 3/60
7397/7397 [==============================] - 0s 53us/step - loss: 1649.5518 - mse: 1649.5516 - mae: 25.5564
Epoch 4/60
7397/7397 [==============================] - 0s 53us/step - loss: 1643.6902 - mse: 1643.6902 - mae: 25.4941
Epoch 5/60
7397/7397 [==============================] - 0s 53us/step - loss: 1639.7708 - mse: 1639.7711 - mae: 25.4351
Epoch 6/60
7397/7397 [==============================] - 0s 53us/step - loss: 1632.6832 - mse: 1632.6838 - mae: 25.4457
Epoch 7/60
7397/7397 [==============================] - 0s 53us/step - loss: 1634.7288 - mse: 1634.7284 - mae: 25.4245
Epoch 8/60
7397/7397 [==============================] - 0s 53us/step - loss: 1628.1342 - mse: 1628.1338 - mae: 25.4678
Epoch 9/60
7397/7397 [=========================

RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f3d4241c850>, as the constructor either does not set or modifies parameter optimizer

In [7]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_
#results = rnd_search_cv.cv_results_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))
#print("results:".format(results))

the best parameters are:{'optimizer': <keras.optimizers.Nadam object at 0x7f3d74f75310>, 'n_neurons': 80, 'n_hidden': 2, 'kernel_initializer': 'he_normal', 'bias_initializer': <keras.initializers.Ones object at 0x7f3d4c4e8650>}
the best score is:-2072.7588348385707
